In [1]:
!pip install pandas scikit-learn joblib
!pip install geopandas
!pip install scikit-learn

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# Función para preparar los datos diarios
def preparar_datos(fecha_inicio):
    # Crear un rango de fechas de seis meses desde la fecha de inicio
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fechas = pd.date_range(start=fecha_inicio, periods=6 * 30, freq='D')
    
    # Crear un DataFrame con las fechas y datos simulados
    datos = pd.DataFrame({'datetime': fechas})
    datos['year'] = datos['datetime'].dt.year
    datos['month'] = datos['datetime'].dt.month
    datos['day'] = datos['datetime'].dt.day

    # Simular datos de temperatura, humedad y precipitación
    datos['temp'] = 20 + 5 * np.sin(datos.index * 0.1)
    datos['humidity'] = 50 + 10 * np.cos(datos.index * 0.1)
    datos['precip'] = np.random.rand(len(datos))

    return datos

# Función para entrenar y devolver los modelos y predicciones
def entrenar_predecir(fecha_inicio):
    datos_intervalo = preparar_datos(fecha_inicio)

    # Seleccionar las características relevantes
    X = datos_intervalo[['temp', 'humidity', 'precip']].fillna(0)

    # Variables objetivo
    y_humedad = datos_intervalo['humidity']
    y_precip = datos_intervalo['precip']
    y_drought = (datos_intervalo['precip'] < 1).astype(int)

    # Dividir el conjunto de datos
    X_train, X_test, y_train_humedad, _ = train_test_split(X, y_humedad, test_size=0.2, random_state=42)
    X_train_precip, X_test_precip, y_train_precip, _ = train_test_split(X, y_precip, test_size=0.2, random_state=42)
    X_train_drought, X_test_drought, y_train_drought, _ = train_test_split(X, y_drought, test_size=0.2, random_state=42)

    # Entrenar los modelos
    modelo_humidity = RandomForestRegressor(n_estimators=100, random_state=42)
    modelo_humidity.fit(X_train, y_train_humedad)

    modelo_precip = RandomForestRegressor(n_estimators=100, random_state=42)
    modelo_precip.fit(X_train_precip, y_train_precip)

    modelo_drought = RandomForestRegressor(n_estimators=100, random_state=42)
    modelo_drought.fit(X_train_drought, y_train_drought)

    # Obtener predicciones
    pred_humedad = modelo_humidity.predict(X)
    pred_precip = modelo_precip.predict(X)
    pred_drought = modelo_drought.predict(X)
    
    return datos_intervalo, pred_humedad, pred_precip, pred_drought

# Función para mostrar una tabla de resultados
def crear_tabla_resultados(datos_intervalo, pred_humedad, pred_precip, pred_drought):
    tabla_resultados = pd.DataFrame({
        'Fecha': datos_intervalo['datetime'],
        'Humedad Real': datos_intervalo['humidity'],
        'Humedad Predicción': pred_humedad,
        'Precipitación Real': datos_intervalo['precip'],
        'Precipitación Predicción': pred_precip,
        'Sequía Real': (datos_intervalo['precip'] < 1).astype(int),
        'Sequía Predicción': pred_drought
    })
    
    return tabla_resultados

# Función para recomendar cultivos
def recomendar_cultivo(datos_climaticos, N, P, K, pH):
    # Filtrar o preparar datos para el modelo de recomendación
    X_recomendacion = pd.DataFrame({
        'N': [N],
        'P': [P],
        'K': [K],
        'Temperature': [datos_climaticos['Humedad Real'].mean()],  # Usar media o un valor representativo
        'Humidity': [datos_climaticos['Humedad Real'].mean()],      # Usar media o un valor representativo
        'pH': [pH],
        'Rainfall': [datos_climaticos['Precipitación Real'].mean()] # Usar media o un valor representativo
    })

    # Cargar el modelo de recomendación de cultivos
    modelo_cultivo = joblib.load('modelo_cultivo.pkl')
    
    # Hacer predicciones
    recomendaciones = modelo_cultivo.predict(X_recomendacion)
    return recomendaciones

# Ejemplo para probar el envío de datos con una fecha específica
fecha_prueba = '2019-01-01'  # Cambiar a una fecha de prueba válida
datos_intervalo, pred_humedad, pred_precip, pred_drought = entrenar_predecir(fecha_prueba)

# Crear la tabla de resultados
datos_predicciones = crear_tabla_resultados(datos_intervalo, pred_humedad, pred_precip, pred_drought)

# Ejemplo de uso con valores constantes de nutrientes y pH
N_val, P_val, K_val, ph_val = 90, 40, 40, 6.5
recomendaciones_finales = recomendar_cultivo(datos_predicciones, N_val, P_val, K_val, ph_val)

# Mostrar las recomendaciones finales
print(recomendaciones_finales)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Humidity
- Rainfall
- Temperature
- pH
Feature names seen at fit time, yet now missing:
- humidity
- ph
- rainfall
- temperature


In [10]:
joblib.dump(modelo_cultivo, 'modelo_cultivo.pkl')

# Para cargar el modelo posteriormente
# modelo_cultivo = joblib.load('modelo_cultivo.pkl')


['modelo_cultivo.pkl']

In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Cargar el modelo ya entrenado
# modelo_cultivo = joblib.load('modelo_cultivo.pkl')  # Descomentar si es necesario

# Simulación de datos de entrada (un solo conjunto de valores)
data_prueba = {
    'N': [80],               # Nitrógeno
    'P': [40],               # Fósforo
    'K': [20],               # Potasio
    'temperature': [25],     # Temperatura en grados Celsius
    'humidity': [75],        # Humedad en porcentaje
    'ph': [6.5],             # pH del suelo
    'rainfall': [120]        # Lluvia en mm
}

# Crear un DataFrame a partir de los datos de prueba
df_prueba = pd.DataFrame(data_prueba)

# Realizar las predicciones
simulacion_predicciones = modelo_cultivo.predict(df_prueba)

# Obtener las probabilidades de predicción para cada clase
probs = modelo_cultivo.predict_proba(df_prueba)

# Crear un DataFrame para las probabilidades y los cultivos
df_probs = pd.DataFrame(probs, columns=modelo_cultivo.classes_)

# Obtener las 10 mejores opciones basadas en la máxima probabilidad
top_n = 10
mejores_opciones = df_probs.iloc[0].nlargest(top_n).reset_index()
mejores_opciones.columns = ['Recommended Crop', 'Probability']

# Redondear las probabilidades para mejor visualización
mejores_opciones['Probability'] = mejores_opciones['Probability'].round(4)

# Ordenar por probabilidad de éxito
mejores_opciones = mejores_opciones.sort_values(by='Probability', ascending=False)

# Mostrar los resultados con los datos de cultivo
resultados = pd.DataFrame({
    'N': df_prueba['N'].repeat(top_n).values,
    'P': df_prueba['P'].repeat(top_n).values,
    'K': df_prueba['K'].repeat(top_n).values,
    'Temperature': df_prueba['temperature'].repeat(top_n).values,
    'Humidity': df_prueba['humidity'].repeat(top_n).values,
    'pH': df_prueba['ph'].repeat(top_n).values,
    'Rainfall': df_prueba['rainfall'].repeat(top_n).values,
    'Recommended Crop': mejores_opciones['Recommended Crop'].values,
    'Probability': mejores_opciones['Probability'].values
})

# Imprimir solo las recomendaciones de cultivos (simulación) en orden del mejor al peor
print("Resultados de las recomendaciones de cultivos:")
print(resultados[['Recommended Crop', 'N', 'P', 'K', 'Temperature', 'Humidity', 'pH', 'Rainfall', 'Probability']])

Resultados de las recomendaciones de cultivos:
  Recommended Crop   N   P   K  Temperature  Humidity   pH  Rainfall  \
0            maize  80  40  20           25        75  6.5       120   
1           cotton  80  40  20           25        75  6.5       120   
2             jute  80  40  20           25        75  6.5       120   
3           coffee  80  40  20           25        75  6.5       120   
4       pigeonpeas  80  40  20           25        75  6.5       120   
5           banana  80  40  20           25        75  6.5       120   
6           orange  80  40  20           25        75  6.5       120   
7           papaya  80  40  20           25        75  6.5       120   
8            apple  80  40  20           25        75  6.5       120   
9        blackgram  80  40  20           25        75  6.5       120   

   Probability  
0         0.51  
1         0.26  
2         0.12  
3         0.06  
4         0.02  
5         0.01  
6         0.01  
7         0.01  
8      